# Template Live data

### Import Library

In [39]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests
import pickle
import os
from pathlib import Path

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [40]:
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
]

HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
VNDIRECT_DATA_HISTORY_URL = 'https://dchart-api.vndirect.com.vn/dchart/history'
URL = VNDIRECT_DATA_HISTORY_URL

In [41]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-3)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": str(ticker),
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(URL, params=params, headers=HEADERS)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset

In [42]:
def prepareData(htd):
    if 'Time' in htd.columns:
        from datetime import datetime

        htd['DateStr'] = htd.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    htd['Date'] = pd.to_datetime(htd['DateStr'])
    ticker_data = htd.set_index('Date')
    ticker_data.drop(columns=['Time', 'DateStr'], inplace=True)
    return ticker_data

### Load Price Data

In [43]:
ticker = "VN30F1M"
htd = getStockHistoryData(ticker, 0, 0)
ticker_data = prepareData(htd)
ticker_data = ticker_data.dropna()
ticker_data

,Open,High,Low,Close,Volume
Date,,,,,
2024-08-01 09:00:00,1305.5,1306.0,1305.2,1305.5,6035
2024-08-01 09:05:00,1305.3,1306.2,1305.3,1306.1,2761
2024-08-01 09:10:00,1306.1,1306.1,1305.6,1306.0,1570
2024-08-01 09:15:00,1306.0,1306.2,1304.8,1306.0,2883
2024-08-01 09:20:00,1306.0,1306.1,1304.3,1304.8,2788
...,...,...,...,...,...
2024-11-01 14:15:00,1330.6,1331.1,1326.6,1328.9,13201
2024-11-01 14:20:00,1329.0,1331.7,1328.7,1331.1,8731
2024-11-01 14:25:00,1331.5,1332.1,1329.0,1329.8,10078


In [44]:
def bar9h(tick):
    tick = tick[(tick.index.hour == 9) & (tick.index.minute == 0)]
    if len(tick):
        return tick[0]


def bar14h25(tick):
    tick = tick[(tick.index.hour == 14) & (tick.index.minute == 25)]
    if len(tick):
        return tick[0]
        
def prepare_pivot_data(data):
    data['Close_9h'] = data['Close']
    data['exit_price'] = data['Close']
    day_data = data.resample('D').agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min',
        'Close_9h': bar9h,
        'exit_price': bar14h25
    })
    day_data.dropna(subset=['Close'], inplace=True)
    day_data['High_s'] = day_data['High'].shift(1)
    day_data['Low_s'] = day_data['Low'].shift(1)
    day_data['Close_s'] = day_data['Close'].shift(1)
    #
    day_data['p'] = (day_data['High_s'] + day_data['Low_s'] + day_data['Close_s']) / 3
    day_data['p_rate_x'] = 100 * (day_data.p - day_data.Close_s) / day_data.Close_s
    day_data['p_rate_y'] = (day_data.p - day_data.Close_9h) / (day_data.High_s - day_data.Low_s)
    #
    day_data['r1'] = 2 * day_data['p'] - day_data['Low_s']
    day_data['r1_rate_x'] = 100 * (day_data.r1 - day_data.Close_s) / day_data.Close_s
    day_data['r1_rate_y'] = (day_data.r1 - day_data.Close_9h) / (
                day_data.High_s - day_data.Low_s)
    #
    day_data['r2'] = day_data['p'] + day_data['High_s'] - day_data['Low_s']
    day_data['r2_rate_x'] = 100 * (day_data.r2 - day_data.Close_s) / day_data.Close_s
    day_data['r2_rate_y'] = (day_data.r2 - day_data.Close_9h) / (
                day_data.High_s - day_data.Low_s)
    #
    day_data['r3'] = day_data['p'] * 2 + day_data['High_s'] - 2 * day_data['Low_s']
    day_data['r3_rate_x'] = 100 * (day_data.r3 - day_data.Close_s) / day_data.Close_s
    day_data['r3_rate_y'] = (day_data.r3 - day_data.Close_9h) / (
                day_data.High_s - day_data.Low_s)
    #
    day_data['r4'] = day_data['p'] * 3 + day_data['High_s'] - 3 * day_data['Low_s']
    day_data['r4_rate_x'] = 100 * (day_data.r4 - day_data.Close_s) / day_data.Close_s
    day_data['r4_rate_y'] = (day_data.r4 - day_data.Close_9h) / (
                day_data.High_s - day_data.Low_s)
    #
    day_data['s1'] = 2 * day_data['p'] - day_data['High_s']
    day_data['s1_rate_x'] = 100 * (day_data.s1 - day_data.Close_s) / day_data.Close_s
    day_data['s1_rate_y'] = (day_data.s1 - day_data.Close_9h) / (
                day_data.High_s - day_data.Low_s)
    #
    day_data['s2'] = day_data['p'] - (day_data['High_s'] - day_data['Low_s'])
    day_data['s2_rate_x'] = 100 * (day_data.s2 - day_data.Close_s) / day_data.Close_s
    day_data['s2_rate_y'] = (day_data.s2 - day_data.Close_9h) / (
                day_data.High_s - day_data.Low_s)
    #
    day_data['s3'] = day_data['p'] * 2 - (2 * day_data['High_s'] - day_data['Low_s'])
    day_data['s3_rate_x'] = 100 * (day_data.s3 - day_data.Close_s) / day_data.Close_s
    day_data['s3_rate_y'] = (day_data.s3 - day_data.Close_9h) / (
            day_data.High_s - day_data.Low_s)
    #
    day_data['s4'] = day_data['p'] * 3 - (3 * day_data['High_s'] - day_data['Low_s'])
    day_data['s4_rate_x'] = 100 * (day_data.s4 - day_data.Close_s) / day_data.Close_s
    day_data['s4_rate_y'] = (day_data.s4 - day_data.Close_9h) / (
            day_data.High_s - day_data.Low_s)
    #
    return day_data

In [45]:
day_data = prepare_pivot_data(ticker_data)

In [46]:
day_data

,Open,Close,High,Low,Close_9h,exit_price,High_s,Low_s,Close_s,p,...,s1_rate_y,s2,s2_rate_x,s2_rate_y,s3,s3_rate_x,s3_rate_y,s4,s4_rate_x,s4_rate_y
Date,,,,,,,,,,,,,,,,,,,,,
2024-08-01,1305.5,1275.0,1306.2,1271.0,1305.5,1277.6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-02,1266.2,1275.9,1277.5,1256.3,1264.6,1276.5,1306.2,1271.0,1275.0,1284.066667,...,-0.075758,1248.866667,-2.049673,-0.446970,1226.733333,-3.785621,-1.075758,1204.6,-5.521569,-1.704545
2024-08-05,1256.8,1233.0,1262.8,1227.3,1253.1,1232.8,1277.5,1256.3,1275.9,1269.900000,...,0.433962,1248.700000,-2.131829,-0.207547,1241.100000,-2.727486,-0.566038,1233.5,-3.323144,-0.924528
2024-08-06,1244.7,1249.6,1256.0,1237.2,1247.1,1252.8,1262.8,1227.3,1233.0,1241.033333,...,-0.784038,1205.533333,-2.227629,-1.170892,1183.766667,-3.992971,-1.784038,1162.0,-5.758313,-2.397183
2024-08-07,1252.3,1250.0,1255.5,1245.7,1251.5,1250.4,1256.0,1237.2,1249.6,1247.600000,...,-0.654255,1228.800000,-1.664533,-1.207447,1220.400000,-2.336748,-1.654255,1212.0,-3.008963,-2.101064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-28,1330.0,1332.0,1332.3,1327.2,1329.2,1331.1,1339.8,1327.0,1327.2,1331.333333,...,-0.494792,1318.533333,-0.653004,-0.833333,1310.066667,-1.290938,-1.494792,1301.6,-1.928873,-2.156250
2024-10-29,1334.0,1339.4,1341.0,1334.0,1335.4,1338.2,1332.3,1327.2,1332.0,1330.500000,...,-1.313725,1325.400000,-0.495495,-1.960784,1323.600000,-0.630631,-2.313725,1321.8,-0.765766,-2.666667
2024-10-30,1341.6,1342.0,1342.7,1331.5,1340.9,1336.3,1341.0,1334.0,1339.4,1338.133333,...,-0.804762,1331.133333,-0.617192,-1.395238,1328.266667,-0.831218,-1.804762,1325.4,-1.045244,-2.214286


In [47]:
today = day_data.iloc[-1]

In [48]:
today

Open          1340.400000
Close         1330.400000
High          1340.400000
Low           1326.600000
Close_9h      1337.800000
exit_price    1329.800000
High_s        1347.800000
Low_s         1333.700000
Close_s       1343.500000
p             1341.666667
p_rate_x        -0.136459
p_rate_y         0.274232
r1            1349.633333
r1_rate_x        0.456519
r1_rate_y        0.839243
r2            1355.766667
r2_rate_x        0.913038
r2_rate_y        1.274232
r3            1363.733333
r3_rate_x        1.506017
r3_rate_y        1.839243
r4            1371.700000
r4_rate_x        2.098995
r4_rate_y        2.404255
s1            1335.533333
s1_rate_x       -0.592979
s1_rate_y       -0.160757
s2            1327.566667
s2_rate_x       -1.185957
s2_rate_y       -0.725768
s3            1321.433333
s3_rate_x       -1.642476
s3_rate_y       -1.160757
s4            1315.300000
s4_rate_x       -2.098995
s4_rate_y       -1.595745
Name: 2024-11-01 00:00:00, dtype: float64

In [49]:
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent.parent

In [50]:
def find_mean_return(rate_x, rate_y, pivot_df):
    found_i = None
    for i in pivot_df.index:
        if i.left < rate_x <= i.right:
            found_i = i
            break
    found_c = None
    for c in pivot_df.columns:
        if c.left < rate_y <= c.right:
            found_c = c
            break
    found = pivot_df.loc[pivot_df.index == found_i, found_c]
    if len(found):
        val = found.values[0]
        return val

## At R1 - Analytics

In [53]:
mean_return_at_r1_db_file = str(algo_dir) + '/VN30ps/pivots/v2/r1_mean_return_db.pkl'
is_file = os.path.isfile(mean_return_at_p_db_file)
if is_file:
    with open(mean_return_at_p_db_file, 'rb') as f:
        r1_mean_return_df = pickle.load(f)
        _at_today_r1_mean_return = find_mean_return(today['r1_rate_x'], today['r1_rate_y'], r1_mean_return_df)
        print(_at_today_r1_mean_return)
else:
    print(fibo_db_file)
    print('DB file not found.')

-10.239906094546912


In [54]:
risk_reward_at_r1_db_file = str(algo_dir) + '/VN30ps/pivots/v2/r1_risk_reward_db.pkl'
is_file = os.path.isfile(risk_reward_at_r1_db_file)
if is_file:
    with open(risk_reward_at_r1_db_file, 'rb') as f:
        r1_risk_reward_df = pickle.load(f)
        _at_today_r1_risk_reward = find_mean_return(today['r1_rate_x'], today['r1_rate_y'], r1_risk_reward_df)
        print(_at_today_r1_risk_reward)
else:
    print(fibo_db_file)
    print('DB file not found.')

-29.39999999999941


### At P - Analytics

In [51]:
mean_return_at_p_db_file = str(algo_dir) + '/VN30ps/pivots/v2/p_mean_return_db.pkl'
is_file = os.path.isfile(mean_return_at_p_db_file)
if is_file:
    with open(mean_return_at_p_db_file, 'rb') as f:
        stored_pivot_df = pickle.load(f)
        # sns.heatmap(
        # stored_pivot_df,
        # cmap=sns.diverging_palette(0, 240, s=100, l=55, center="dark", as_cmap=True),
        # annot=True, fmt=".1f");
        _at_today_p_mean_return = find_mean_return(today['p_rate_x'], today['p_rate_y'], stored_pivot_df)
        print(_at_today_p_mean_return)
else:
    print(fibo_db_file)
    print('DB file not found.')

10.10989010989006


In [52]:
risk_reward_at_p_db_file = str(algo_dir) + '/VN30ps/pivots/v2/p_risk_reward_db.pkl'
is_file = os.path.isfile(risk_reward_at_p_db_file)
if is_file:
    with open(risk_reward_at_p_db_file, 'rb') as f:
        stored_rr_pivot_df = pickle.load(f)
        # sns.heatmap(
        # stored_pivot_df,
        # cmap=sns.diverging_palette(0, 240, s=100, l=55, center="dark", as_cmap=True),
        # annot=True, fmt=".1f");
        _at_today_p_mean_risk_reward = find_mean_return(today['p_rate_x'], today['p_rate_y'], stored_rr_pivot_df)
        print(_at_today_p_mean_risk_reward)
else:
    print(fibo_db_file)
    print('DB file not found.')

11.933333333333167


## At S1 - Analytics

In [55]:
mean_return_at_s1_db_file = str(algo_dir) + '/VN30ps/pivots/v2/s1_mean_return_db.pkl'
is_file = os.path.isfile(mean_return_at_p_db_file)
if is_file:
    with open(mean_return_at_p_db_file, 'rb') as f:
        s1_mean_return_df = pickle.load(f)
        _at_today_s1_mean_return = find_mean_return(today['s1_rate_x'], today['s1_rate_y'], s1_mean_return_df)
        print(_at_today_s1_mean_return)
else:
    print(fibo_db_file)
    print('DB file not found.')

1.6747822653440938


In [56]:
risk_reward_at_s1_db_file = str(algo_dir) + '/VN30ps/pivots/v2/s1_risk_reward_db.pkl'
is_file = os.path.isfile(risk_reward_at_s1_db_file)
if is_file:
    with open(risk_reward_at_s1_db_file, 'rb') as f:
        s1_risk_reward_df = pickle.load(f)
        _at_today_s1_risk_reward = find_mean_return(today['s1_rate_x'], today['s1_rate_y'], s1_risk_reward_df)
        print(_at_today_s1_risk_reward)
else:
    print(fibo_db_file)
    print('DB file not found.')

3.199999999999932


## At S2 - Analytics

In [57]:
mean_return_at_s2_db_file = str(algo_dir) + '/VN30ps/pivots/v2/s2_mean_return_db.pkl'
is_file = os.path.isfile(mean_return_at_p_db_file)
if is_file:
    with open(mean_return_at_p_db_file, 'rb') as f:
        s2_mean_return_df = pickle.load(f)
        _at_today_s2_mean_return = find_mean_return(today['s2_rate_x'], today['s2_rate_y'], s2_mean_return_df)
        print(_at_today_s2_mean_return)
else:
    print(fibo_db_file)
    print('DB file not found.')

5.98697642617994


In [58]:
risk_reward_at_s2_db_file = str(algo_dir) + '/VN30ps/pivots/v2/s2_risk_reward_db.pkl'
is_file = os.path.isfile(risk_reward_at_s2_db_file)
if is_file:
    with open(risk_reward_at_s2_db_file, 'rb') as f:
        s2_risk_reward_df = pickle.load(f)
        _at_today_s2_risk_reward = find_mean_return(today['s2_rate_x'], today['s2_rate_y'], s2_risk_reward_df)
        print(_at_today_s2_risk_reward)
else:
    print(fibo_db_file)
    print('DB file not found.')

36.57500000000016
